In [ ]:
!pip install adapters datasets transformers torch
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, EvalPrediction
from adapters import AutoAdapterModel, AdapterConfig, AdapterTrainer

# 1. Load Model and Tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoAdapterModel.from_pretrained(model_name)

# 2. Add Sentiment Adapter
# This adds the bottleneck layers and a classification head
model.add_adapter("sentiment_adapter", config="pfeiffer")
model.add_classification_head("sentiment_adapter", num_labels=2)

# 3. Freeze base model and activate adapter for training
model.train_adapter("sentiment_adapter")

In [ ]:
# 4. Dataset Prep (using IMDB)
dataset = load_dataset("stanfordnlp/imdb")
def encode_batch(batch):
    return tokenizer(batch["text"], max_length=128, truncation=True, padding="max_length")

tokenized_ds = dataset.map(encode_batch, batched=True)

# 5. Training Setup
args = TrainingArguments(
    output_dir="./adapter_sentiment",
    learning_rate=1e-4, # Adapters usually need a higher LR than full fine-tuning
    num_train_epochs=3,
    per_device_train_batch_size=32,
    remove_unused_columns=False,
)


trainer = AdapterTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
)

trainer.train()
model.save_adapter("./my_final_adapter", "sentiment_adapter")

In [ ]:
from adapters import AutoAdapterModel
from transformers import AutoTokenizer

model = AutoAdapterModel.from_pretrained("roberta-base")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Load your custom-trained adapter
model.load_adapter("./my_final_adapter")
model.set_active_adapters("sentiment_adapter")

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return "Positive" if predictions.item() == 1 else "Negative"

print(analyze_sentiment("This movie was an absolute masterpiece of modern cinema!"))